In [5]:
"""
Reassemble a JPEG image from images sent by satellite-connected timelapse camera
JPEG images are named with the timestamp of when the image was captured
Change directory variable based on where the images are stored - just dump all the messages in here

Note: This has no protection strategy for when an image has not been completely transmitted - 
that is, not all the messages have arrived. The script will attempt to assemble a JPEG and throw a FileNotFoundException
This should be relatively easy to fix - add catch that assumes FileNotFound is a failure of that image 
Need a mechanism to not immediately try to assemble that image again (keep track of IDs that have been tried?)

Author: Rachel Pagdin
July 16, 2024
"""

import os
import re

directory = 'jpeg-test-deployment/'

regex = re.compile(r'\d{8}.BIN')      #this is the format for image messages from the timelapse camera
names = '|'.join(os.listdir(directory))
match = regex.search(names)

while match is not None:
    name = match.group()       #convert to string
    
    # extract info about image from title
    image_id = int(name[0:2])
    totseq = int(name[5:8])
    
    filename = directory + name
    
    # get timestamp information from the header of that file
    with open(filename, 'rb') as f:
        contents = f.read()
        year = contents[7]
        month = contents[8]
        day = contents[9]
        hour = contents[10]
        
    
    # iterate through the messages for this image
    image_data = []
    for seq in range(1, totseq+1):
        filename = '{}{:02d}{:03d}{:03d}.bin'.format(directory, image_id, seq, totseq)
    
        f = open(filename, 'rb')
        contents = f.read()
        f.close()
    
        temp = [b for b in contents]
    
        image_data.extend(temp[14::])
        
        
    # write the data to a new jpg file
    image_name = '{}/{:02d}{:02d}{:02d}{:02d}.jpg'.format(directory, year, month, day, hour)
    f = open(image_name, 'wb')
    f.write(bytes(image_data))
    f.close()    
    
        
    # rename the messages that were used (assume transfer to image was successful)
    # could instead delete the files
    for seq in range(1, totseq+1):
        filename = '{}{:02d}{:03d}{:03d}.bin'.format(directory, image_id, seq, totseq)
        newname = '{}{:02d}{:03d}{:03d}-used.bin'.format(directory, image_id, seq, totseq)
        os.rename(filename, newname)        #indicate this file has already been used
        
    names = '|'.join(os.listdir(directory))
    match = regex.search(names)      # search for another match (i.e. another image)